In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime
import numpy as np
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import folium
import time
from bs4 import BeautifulSoup
import requests
import json
import random
from tqdm import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
data_weather = pd.read_csv('weatherAUS.csv')

In [3]:
data_weather.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [4]:
data_weather.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,143975.000000,144199.000000,142199.000000,82670.000000,75625.000000,135197.000000,143693.000000,142398.000000,142806.000000,140953.000000,130395.00000,130432.000000,89572.000000,86102.000000,143693.000000,141851.00000
mean,12.194034,23.221348,2.360918,5.468232,7.611178,40.035230,14.043426,18.662657,68.880831,51.539116,1017.64994,1015.255889,4.447461,4.509930,16.990631,21.68339
std,6.398495,7.119049,8.478060,4.193704,3.785483,13.607062,8.915375,8.809800,19.029164,20.795902,7.10653,7.037414,2.887159,2.720357,6.488753,6.93665
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.50000,977.100000,0.000000,0.000000,-7.200000,-5.40000
25%,7.600000,17.900000,0.000000,2.600000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.90000,1010.400000,1.000000,2.000000,12.300000,16.60000
50%,12.000000,22.600000,0.000000,4.800000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.60000,1015.200000,5.000000,5.000000,16.700000,21.10000
75%,16.900000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.40000,1020.000000,7.000000,7.000000,21.600000,26.40000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.00000,1039.600000,9.000000,9.000000,40.200000,46.70000


In [5]:
data_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [6]:
data_weather['Date'] = pd.to_datetime(data_weather['Date'], format='%Y-%m-%d')
data_weather = data_weather[data_weather['RainToday'].notna()]
data_weather = data_weather[data_weather['RainTomorrow'].notna()]
# data_weather['RainToday'].replace({'No':0, 'Yes': 1}, inplace = True)
# data_weather['RainTomorrow'].replace({'No':0, 'Yes': 1}, inplace = True)

In [7]:
# Getting latitude&longitude of locations

locs = data_weather.Location.unique()
locs.sort()
api = "http://api.positionstack.com/v1/forward"
token = 'f898acba181f26412807e4fc0bf8c2ea'
locs_df = pd.DataFrame(locs, columns=['Location'])
locs_df['latitude'] = ''
locs_df['longitude'] = ''
for ind, row in locs_df.iterrows():
    try:
        query = row['Location'] + ', Australia'
        url = f'{api}?access_key={token}&query={query}&limit=1'
        page = requests.get(url)
        doc = BeautifulSoup(page.content, 'html.parser')
        json_dict = json.loads(str(doc))
        locs_df.loc[int(ind), 'latitude'] = str(json_dict['data'][0]['latitude'])
        locs_df.loc[int(ind), 'longitude'] = str(json_dict['data'][0]['longitude'])
    except:
        locs_df.loc[int(ind), 'latitude'] = '-33.865143'
        locs_df.loc[int(ind), 'longitude'] = '151.209900'


In [8]:
locs_df

,Location,latitude,longitude
0,Adelaide,-34.929075,138.602578
1,Albany,-35.031075,117.89494
2,Albury,-36.074771,146.914504
3,AliceSprings,-33.865143,151.209900
4,BadgerysCreek,-33.865143,151.209900
5,Ballarat,-37.563892,143.852395
6,Bendigo,-36.757608,144.27957
7,Brisbane,-27.469309,153.026164
8,Cairns,-16.922929,145.775689
9,Canberra,-35.470057,148.954507


# Combining all the code

In [9]:
def str_to_num(data, col):
    mapdict = {}
    values = list(data[col].unique())
    for i in range(len(values)):
        mapdict[values[i]] = i
    return mapdict

def clean_data(data):
    n = len(data)
    data = data.drop(['Location'], axis = 1)
    for col in data.columns:
        b = round(data[col].isna().sum()/n,2)
        if (b>= 0.1):
            data = data.drop([col], axis = 1)
        else:
            try:
                data[col].fillna((data[col].mean()), inplace=True)
            except:
                pass
            if (data[col].dtype == 'O'):
                data[col].replace(str_to_num(data, col),inplace = True)
    data = data.dropna()
    return data

def get_RFC(locaction):
    try:
        data = data_weather[data_weather['Location'] == locaction]
        df = clean_data(data)
        in_sample = df[df.Date.dt.year != 2017]
        out_of_sample = df[df.Date.dt.year == 2017]

        X=in_sample.drop(['RainTomorrow', 'Date'], axis = 1)
        y=in_sample['RainTomorrow']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        clf = RandomForestClassifier(n_estimators=100)
        clf.fit(X_train,y_train)

        y_pred = clf.predict(X_test)
        test_error = round(metrics.accuracy_score(y_test, y_pred),2)

        X_out = out_of_sample.drop(['RainTomorrow', 'Date'], axis = 1)
        y_out = out_of_sample['RainTomorrow']

        y_fc = clf.predict(X_out)
        forecast_error = round(metrics.accuracy_score(y_out, y_fc),2)
        print(f'Test Accuracy for {locaction} is: {test_error}')
        print(f'Forecast Accuracy for {locaction} is: {forecast_error}')
        df_final = out_of_sample
        df_final['Forecast'] = y_fc
        df_final['FC_accuracy'] = forecast_error
        df_final['Test_accuracy'] = test_error
    except:
        pass

    return df_final
    
    

In [10]:
collect = []
for loc in tqdm(locs_df['Location']):
    try:     
        df_with_results = get_RFC(loc)
        df_with_results['Location'] = str(loc)
        df_with_results['Latitude'] = float(locs_df[locs_df['Location'] == loc]['latitude'].values)
        df_with_results['Longitude'] = float(locs_df[locs_df['Location'] == loc]['longitude'].values)
        df_with_results['Date'] = df_with_results.Date.dt.date
        df_with_results = df_with_results[['Date', 'Location', 'Latitude', 'Longitude', 'RainTomorrow', 'Forecast', 'FC_accuracy', 'Test_accuracy']]
        collect.append(df_with_results)
    except:
        pass

df_final_result = pd.concat(collect, axis=0, ignore_index=True).reset_index(drop = True)



  2%|▉                                           | 1/49 [00:00<00:20,  2.39it/s]

Test Accuracy for Adelaide is: 0.87
Forecast Accuracy for Adelaide is: 0.92


  4%|█▊                                          | 2/49 [00:00<00:18,  2.57it/s]

Test Accuracy for Albany is: 0.81
Forecast Accuracy for Albany is: 0.85


  6%|██▋                                         | 3/49 [00:01<00:17,  2.63it/s]

...

Test Accuracy for Witchcliffe is: 0.87
Forecast Accuracy for Witchcliffe is: 0.86


 98%|██████████████████████████████████████████ | 48/49 [00:18<00:00,  2.51it/s]

Test Accuracy for Wollongong is: 0.86
Forecast Accuracy for Wollongong is: 0.78


100%|███████████████████████████████████████████| 49/49 [00:18<00:00,  2.66it/s]

Test Accuracy for Woomera is: 0.94
Forecast Accuracy for Woomera is: 0.96


In [11]:
df_final_result.head()

,Date,Location,Latitude,Longitude,RainTomorrow,Forecast,FC_accuracy,Test_accuracy
0,2017-01-01,Adelaide,-34.929075,138.602578,0,0,0.92,0.87
1,2017-01-02,Adelaide,-34.929075,138.602578,0,0,0.92,0.87
2,2017-01-03,Adelaide,-34.929075,138.602578,0,0,0.92,0.87
3,2017-01-04,Adelaide,-34.929075,138.602578,0,0,0.92,0.87
4,2017-01-05,Adelaide,-34.929075,138.602578,0,0,0.92,0.87


In [40]:
df_final_result.to_csv('Australia_forecast.csv',index=False)